In [ ]:
!pip install gensim emoji nltk tqdm seaborn torch torchsummary -q

In [1]:
import string

import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary

import gensim
from IPython.display import clear_output

import torchvision
from torchvision import transforms

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import gc
import os

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [12]:
MODE = "local" # valid modes are 'local', 'colab' and 'kaggle'
SAMPLE_NUMBERS = 4
SEQUENCE_LEN = 256
TEST_PORTION = 0.2

In [13]:
if   MODE == "local":
    BASE_PATH = "."
elif MODE == "colab":
    BASE_PATH = "/content/drive/MyDrive/Deep Neural Network - UT/CA 04"
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print(f"Unknown mode {MODE}")
    exit(1)

In [14]:
DATA_PATH = os.path.join(BASE_PATH, "twitter-suicidal_data.csv")

W2V_PATH = os.path.join(BASE_PATH, "w2v_vectors.kv")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Data Exploration

## Load Data

In [15]:
df = pd.read_csv(DATA_PATH)

## Data statistics

In [16]:
df.head(5)

,tweet,intention
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9119 entries, 0 to 9118
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      9119 non-null   object
 1   intention  9119 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 142.6+ KB


In [ ]:
print(f"Number of samples: {len(df)}")

In [ ]:
intentions = df["intention"].value_counts()

plt.figure(figsize=(7, 5))
sns.barplot(x=intentions.index, y=intentions.values)
plt.title("Intentions Distribution")
plt.show()

In [18]:
from sklearn.utils import compute_class_weight

labels = df["intention"]
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
weights = torch.tensor(class_weights, dtype=torch.float32, device=DEVICE)
print(class_weights)
del labels

[0.89035345 1.14044522]


# Data Preprocessing

In [ ]:
import nltk
import emoji
import re

nltk.download(["stopwords", "punkt", "wordnet", "averaged_perceptron_tagger"])

In [ ]:
def convert_emoji_to_text(text):
    return emoji.demojize(text, delimiters=(" ", " "))

stopwords = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.WordNetLemmatizer()

def nltk_pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return 'a'
    elif nltk_tag.startswith('V'):
        return 'v'
    elif nltk_tag.startswith('N'):
        return 'n'
    elif nltk_tag.startswith('R'):
        return 'r'
    else:          
        return 'n'

def preprocess_data(text: str):
    """
    Preprocessing steps are as follows:
    0. concatenation of the text (not necessary)
    1. lowercase the text
    2. remove punctuation
    3. remove numbers
    4. remove urls
    5. remove usernames
    6. remove extra spaces
    7. convert emojis to text
    8. remove non-word characters
    9. lemmatization and tokenization of the text
    10. remove stopwords
    :param text: str
    :return: tokens: list[str]
    """
    # text = ''.join(text)
    # lowercase the text
    text = text.lower()
    # remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    # remove numbers
    text = re.sub(r"\d+", "", text)
    # remove urls,
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # remove usernames
    text = re.sub(r'@\S+', '', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # convert emojis to text
    text = convert_emoji_to_text(text)
    # remove non-word characters
    text = re.sub(r"[^\w\s]", "", text)
    # lemmatization and tokenization of the text
    tokenized = nltk.word_tokenize(text)
    pos_tag = nltk.pos_tag(tokenized)
    tokens = [
        lemmatizer.lemmatize(token, nltk_pos_tagger(tag))
        for token, tag in pos_tag
    ]
    # remove stopwords
    tokens = [token for token in tokens if token not in stopwords]
    
    return tokens
    

In [ ]:
def format_length_bound(text: str, length:int):
    if len(text) <= length:
        return text
    mm = length // 2 - 3
    rr = length - mm - 5
    return text[:mm] + "|...|" + text[-rr:]
    
def draw_sample_processing(dataframe: pd.DataFrame, sample_numbers: int = SAMPLE_NUMBERS, show=True):
    sample_text = dataframe.sample(sample_numbers)
    label_rawT_procT = [
        (label, tweet, preprocess_data(tweet)) 
        for tweet, label in zip(sample_text["tweet"], sample_text["intention"])
    ]
    if show:
        for idx, (label, rawT, procT) in enumerate(label_rawT_procT, start=1):
            procT = str(procT)
            print(f"Sample {idx}:")
            print(f":::::::Label          : {label}")
            print(f":::::::Raw Tweet      : {format_length_bound(rawT, 50):<50} ({len(rawT)})")
            print(f":::::::Processed Tweet: {format_length_bound(procT, 50):<50} ({len(procT)})")
        return 
    return pd.DataFrame(label_rawT_procT, columns=["Intention", "Raw Tweet", "Processed Tweet"])

In [ ]:
draw_sample_processing(df, show=True)

# Word2Vec - Word Embedding

In [ ]:
# print available word2vec models
import gensim.downloader as api
print(list(api.info()['models'].keys()))

In [ ]:
if os.path.exists(W2V_PATH):
    print("Loading Word2Vec model...")
    w2v_model = gensim.models.KeyedVectors.load(W2V_PATH, mmap='r')
else:
    print("Downloading Word2Vec model...")
    w2v_model = api.load("word2vec-google-news-300")
    print("Saving Word2Vec model...")
    w2v_model.save(W2V_PATH)
    

In [ ]:
EMBEDDING_VECTOR_DIM = w2v_model.vector_size

# Dataset

In [ ]:
class Twitter(Dataset):
    def __init__(self, dataframe: pd.DataFrame, w2v_model: gensim.models.KeyedVectors, aggregate=True, **kw_args):
        self.dataframe = dataframe
        self.w2v_model = w2v_model
        self.aggregate = aggregate
        self.max_sequence_len = kw_args.get("sequence_len", 512)
        if not aggregate and "sequence_len" not in kw_args:
            print("You Should provide 'sequence_len' to use not aggregate option!")
            exit(1)

        self.len = len(self.dataframe)
        self.vector_size = w2v_model.vector_size
        self.lazy_mode = kw_args.get("lazy", True)

        self._proc_dataset()

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        if self.lazy_mode:
            vector = self._get_word_vectors(self.dataframe.iloc[idx]["tokens"])
            return vector, self.dataframe.iloc[idx]["intention"]
        return self.dataframe.iloc[idx]["vector"], self.dataframe.iloc[idx]["intention"]

    def get_vector_size(self):
        return self.vector_size

    def _proc_dataset(self):
        self.dataframe["tokens"] = self.dataframe["tweet"].map(preprocess_data)
        
        # delete samples with empty tokens
        lwz = len(self.dataframe)
        self.dataframe = self.dataframe[self.dataframe["tokens"].map(len) > 0]
        self.dataframe.reset_index(drop=True, inplace=True)
        print(f"Deleted 0-Len Samples: {lwz - len(self.dataframe)}")
        self.seq_report()

        if not self.aggregate:
            self.dataframe["tokens"] = self.dataframe["tokens"].map(self._pad)

        if not self.lazy_mode:
            self.dataframe["vector"] = self.dataframe["tokens"].map(self._get_word_vectors)

    def _get_word_vectors(self, tokens: list) -> torch.tensor:
        if not self.aggregate:
            return torch.stack([
                torch.tensor(self.w2v_model[token] if token in self.w2v_model else np.zeros(self.vector_size), dtype=torch.float32) 
                for token in tokens
            ])
        wv = np.zeros(self.vector_size)
        vc = 0
        for token in tokens:
            if token in self.w2v_model:
                wv += self.w2v_model[token]
                vc += 1
        wv = torch.tensor(wv / max(vc, 1), dtype=torch.float32)
        return wv

    def _pad(self, tokens: list):
        if len(tokens) >= self.max_sequence_len:
            return tokens[:self.max_sequence_len]
        return tokens + [" "] * (self.max_sequence_len - len(tokens))

    def seq_report(self):
        length_all = self.dataframe["tokens"].map(len).tolist()
        max_length = np.max(length_all)
        print(f"Sequence Length Report")
        print(f":::::MAX  LENGTH:::[{max_length:^5}]")
        print(f":::::MIN  LENGTH:::[{np.min(length_all):^5}]")
        print(f":::::MEAN LENGTH:::[{np.mean(length_all):^5}]")

        all_tokens = set()
        for token_set in self.dataframe["tokens"].tolist():
            all_tokens = all_tokens.union(set(token_set))
        unique_tokens_count = len(all_tokens)
        valid_tokens = sum(1 if token in self.w2v_model else 0 for token in all_tokens)
        print("Sequence Tokenization Report")
        print(f":::::All Unique Tokens:::[{unique_tokens_count:^6}")
        print(f":::::All Valid Tokens:::[{valid_tokens:^6}")
        print(f":::::Valid Tokens:::[{round(100*valid_tokens/unique_tokens_count, 2):^5}%]")
        
        print()

    @staticmethod
    def _to_tensor(tokens: list):
        return torch.tensor(tokens, dtype=torch.float32)

# Prepare Data

## Split Data into train-valid

In [ ]:
train_df, valid_df = train_test_split(df, test_size=TEST_PORTION, random_state=42, stratify=df["intention"])

train_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)

del df
gc.collect()

## Create Datasets

In [ ]:
train_dataset = Twitter(train_df, w2v_model, aggregate=False, sequence_len=SEQUENCE_LEN)
valid_dataset = Twitter(valid_df, w2v_model, aggregate=False, sequence_len=SEQUENCE_LEN)

print(f"Train dataset length: {len(train_dataset)}")
print(f"Valid dataset length: {len(valid_dataset)}")


In [ ]:
# train_dataset.seq_report()
# print()
# valid_dataset.seq_report()

# Model and Train

## Configs

In [ ]:
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-2
BATCH_SIZE = 32
EPOCHS = 10

## Utils

In [ ]:
@torch.no_grad()
def model_eval(model, loader, loss_function, device):
    """Returns test_loss, test_acc"""
    test_loss = 0.0
    test_acc = 0.0
    total_samples = 0
    if device == "auto":
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    model = model.to(device)
    itr = tqdm(loader, total=len(loader), leave=False)

    for inputs, labels in itr:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        
        sample_count = len(inputs)
    
        test_loss += loss.item()
        
        _, preds = torch.max(outputs, 1)
        test_correct = torch.sum(preds == labels).item()
        test_acc += test_correct

        itr.set_description("(Eval)")
        itr.set_postfix(
            loss=round(loss.item(), 5),
            accuracy=round(test_correct/sample_count, 5),
            
            )
        total_samples += sample_count

    test_loss = test_loss / len(loader)
    test_acc = test_acc / total_samples

    return test_loss, test_acc


In [ ]:
def train_model(
        model,
        batch_size,
        loss_function,
        optimizer,
        epochs,
        train_set,
        valid_set,
        device,
):
    
    if device == "auto":
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    
    train_losses = []
    train_accs = []
    
    valid_losses = []
    valid_accs = []
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
    
    model.to(device)
    
    itr = tqdm(train_loader, total=len(train_loader), leave=False)
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        epoch_acc = 0
        epoch_samples = 0
        for idx, (inputs, labels) in enumerate(itr, start=1):
            epoch_samples += len(inputs)
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            
            _, preds = torch.max(outputs, 1)
            epoch_acc += torch.sum(preds == labels).item()
        
            loss = loss_function(outputs, labels)
            epoch_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()
            
            itr.set_description(f"(Training) Epoch [{epoch + 1}/{epochs}]")
            itr.set_postfix(
              loss=round(loss.item(), 5),
              accuracy=round(epoch_acc/epoch_samples, 5),
              )
        
        epoch_loss = epoch_loss / len(train_loader)
        
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc/epoch_samples)
        
        model.eval()
        valid_loss, valid_acc = model_eval(
            model=model,
            loader=valid_loader,
            loss_function=loss_function,
            device=device
            )
        valid_losses.append(valid_loss)
        valid_accs.append(valid_acc)
    
    history = {
      "train_loss": train_losses,
      "train_acc": train_accs,
    
      "valid_loss": valid_losses,
      "valid_acc": valid_accs,
    }
    return history

In [ ]:
def trend_plot_helper(pobj):
    plt.figure(figsize=(5*len(pobj), 5))
    for idx, (titler, plots) in enumerate(pobj.items(), start=1):
        plt.subplot(1, len(pobj), idx)
        for label, trend in plots:
            plt.plot(range(1, len(trend)+1), trend, label=label)
        yt, xt = titler.split(' - ')
        plt.xlabel(xt)
        plt.ylabel(yt)
        plt.legend()

## LSTM

In [ ]:
class DetectorLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.1):
        super(DetectorLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

### Train modes

In [ ]:
# Train
lstm_1l_model = DetectorLSTM(input_size=EMBEDDING_VECTOR_DIM, hidden_size=SEQUENCE_LEN, num_layers=1, num_classes=2)
# summary(lstm_1l_model, (EMBEDDING_VECTOR_DIM, SEQUENCE_LEN))

In [ ]:
lstm_1l_model_train_history = train_model(
    model=lstm_1l_model,
    batch_size=BATCH_SIZE,
    loss_function=nn.CrossEntropyLoss(weight=weights),
    optimizer=torch.optim.Adam(lstm_1l_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY),
    epochs=EPOCHS,
    train_set=train_dataset,
    valid_set=valid_dataset,
    device=DEVICE
)

In [ ]:
trend_plot_helper(
    {
        "Accuracy - Epoch": [
            ("Train Acc", lstm_1l_model_train_history["train_acc"]),
            ("Validation Acc", lstm_1l_model_train_history["valid_acc"]),
        ],
        "Loss - Epoch": [
            ("Train Loss", lstm_1l_model_train_history["train_loss"]),
            ("Validation Loss", lstm_1l_model_train_history["valid_loss"])
        ]
    }
)


In [ ]:
del lstm_1l_model_train_history, lstm_1l_model

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

## 2-Layer LSTM

In [ ]:
lstm_2l_model = DetectorLSTM(input_size=EMBEDDING_VECTOR_DIM, hidden_size=SEQUENCE_LEN, num_layers=2, num_classes=2)
# summary(lstm_2l_model, (EMBEDDING_VECTOR_DIM, SEQUENCE_LEN))

In [ ]:
lstm_2l_model_train_history = train_model(
    model=lstm_2l_model,
    batch_size=BATCH_SIZE,
    loss_function=nn.CrossEntropyLoss(weight=weights),
    optimizer=torch.optim.Adam(lstm_2l_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY),
    epochs=EPOCHS,
    train_set=train_dataset,
    valid_set=valid_dataset,
    device=DEVICE
)

In [ ]:
trend_plot_helper(
    {
        "Accuracy - Epoch": [
            ("Train Acc", lstm_2l_model_train_history["train_acc"]),
            ("Validation Acc", lstm_2l_model_train_history["valid_acc"]),
        ],
        "Loss - Epoch": [
            ("Train Loss", lstm_2l_model_train_history["train_loss"]),
            ("Validation Loss", lstm_2l_model_train_history["valid_loss"])
        ]
    }
)


In [ ]:
del lstm_2l_model_train_history, lstm_2l_model

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

## CNN+2-Layer-LSTM

In [ ]:
class DetectorCNNLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.1):
        super(DetectorCNNLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        self.lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.cnn(x)
        x = x.permute(0, 2, 1)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out